In [1]:
%run etl.py

In [ ]:
! unzip -q data/song-data.zip

In [ ]:
! unzip -q data/log-data.zip -d log_data

In [2]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, row_number, udf, dayofweek, col, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql.types import TimestampType

config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [6]:
df=spark.read.parquet("analytics/songplays/*/*/*.parquet")
df.show(5)

+-------------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|  songplay_id|          start_time|user_Id|level|           song_id|         artist_id|session_id|            location|          user_agent|
+-------------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|1099511627776|2018-11-21 21:56:...|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|
|1503238553600|2018-11-24 03:48:...|     88| paid|SONWXQJ12A8C134D94|ARNF6401187FB57032|       888|Sacramento--Rosev...|"Mozilla/5.0 (Mac...|
| 266287972353|2018-11-17 15:37:...|     73| paid|SOBONFF12A6D4F84D8|ARIK43K1187B9AE54C|       518|Tampa-St. Petersb...|"Mozilla/5.0 (Mac...|
|  94489280512|2018-11-11 20:19:...|     80| paid|SOWTBJW12AC468AC6E|ARQGYP71187FB44566|       435|Portland-South Po...|"Mozilla/5.0 (Mac...|
| 9964